# 注意力分数

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

首先我们需要实现一个`sequence_mask`函数，用于将无效的、用于填充的（padding）部分用一个特定的值覆盖掉。这在之后的transformer模型用也经常使用。  
这个函数的基本思想是，传入一个`tensor X`（2D tensor，这里要注意，一般我们的tensor是3D，包括`batch_size`,`key_num` or `guery_num`, `sequence_len`,因此我们会先把tensor按`sequence_len`压缩成2D再传入`sequence_mask`中，`sequence_mask`只处理单个`batch_size`的tensor也就是2D tensor）  
下一个参数`valid_lens`是一个列表，里面包含tensor中每一行，也就是每一个单独的key或query的有效长度，超出这个长度的用第三个参数`value`填充  
下面，我们格局具体函数来详细讲解函数的实现细节


In [2]:
def sequence_mask(X, valid_lens, value=0):
    maxlen = X.shape[1]
    mask = torch.arange(maxlen, dtype=torch.float32, device=X.device)[None,:] < valid_lens[:,None]
    X[~mask] = value
    return X

首先，`maxlen = X.shape[1]`用于获取序列长度，因为传入的是一个2dim的tensor，`shape[1]`就表示,这个非常好理解，比较难理解的是下一行代码  
```python
mask = torch.arange(maxlen, dtype=torch.float32, device=X.device)[None,:] < valid_lens[:,None]
```
这行代码非常好的展现了python的易用性和强大的功能。首先要清楚这行代码的最终目的是生成一个全是布尔值的和输入的X大小相同的且在有效长度内值为`True`，有效长度外值为`False`的tensor，为后面填充value做准备。  
为了更好的理解，我们用一个具体的例子来解释如何实现：  
假设X是一个`3x4`的tensor，  
`valide_lens=[1.,2.,3.]`  
则:`maxlen = 4`  
`torch.arange(maxlen, dtype=torch.float32, device=X.device)`这行代码生成了一个maxlen长度的序列，数据格式是`torch.float32`，为了保证数据和X在同一个decive上方便后续的计算加上`device=X.device`。  
因此我们会得到这样一个tensor：`[0.,1.,2.,3.]`  
而后面的`[None,:]`用来增加维度，None在哪个维度上就在哪个位置增加一个维度，原先我们的tensor是1 dim的，`[None,:]`就表示在第0维的位置增加一个维度，于是我们的tensor变成了`1x4`的一个矩阵`[[0.,1.,2.,3.]]`  
同样的，`<`后面`valid_lens[:,None]`就表示在valid_lens的第1维增加一个维度，于是`valid_lens`就变成了一个`3x1`矩阵`[[1.],[2.],[3.]]`这样我们再来比较大小  
那么问题来了，一个`1x3`的一个矩阵要如何和一个`3x1`矩阵比较大小呢？这就要提到pytorch一个**广播（broadcast）**机制，具体规则如下：  
1. 如果两个张量维度数量不同，在维度较少的张量前面补1，直到维度相同
   * 对于我们的两个张量矩阵都是2 dim已经满足这一条件，跳过
2. 从后往前逐个比较两个张量维度大小：
   1. 若维度大小相同，则该维度保持不变
   2. 如果其中一个张量的维度大小是1，将其复制扩展以匹配另一个张量的维度大小
   3. 如果维度大小不同且没有一个是1，则会报错

我们的两个张量矩阵就很好的利用了这一特性对这两个tensor从左到右看，经过广播后都变成3x3的矩阵：
```
[[0.,1.,2.,3.],
 [0.,1.,2.,3.],
 [0.,1.,2.,3.]]
```
```
valid_lens=[[1.,1.,1.,1.],
            [2.,2.,2.,2.],
            [3.,3.,3.,3.]]
```
这样进行比较，得到最终的mask：
```
mask = [[True,False,False,False],
        [True,True, False,False],
        [True,True, True, False]]
```
最后一行`X[~mask] = value`也很巧妙：  
`~mask`将mask反转（True变False，False变True），`X[~mask]`将X在mask中位置维True的值改成value，从而最终实现mask的功能。

下面，masked_softmax函数实现，将我们最初的3 dim的X转化为2 dim的tensor，对输入valid_lens进行必要处理后输入sequence_mask函数，最终将X还原为3 dim后经过softmax函数处理再输出

In [3]:
def masked_softmax(X, valid_lens):
    if valid_lens is None:
        return nn.functional.softmax(X, dim = -1)
            #如果没有指定valid_lens即无需进行mask，就直接对最后一个维度进行softmax后输出
    else:
        shape = X.shape
            #记录X的形状，获取序列长度和后面还原需要
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
                #如果valid_lens是一维的，则说明是按照每一个bach_size来指定有效长度，即每一个batch_size内的有效长度都相同，因此，用repeat_interleave(valid_lens, shape[1])将长度复制扩展和每一行对应，保证输入sequence_mask的valid_lens格式正确
        else: 
            valid_lens = valid_lens.reshape(-1)
                #如果valid_lens不是一维，则说明已经按照行标注好，只需reshape成一维tensor即可
        X = sequence_mask(X.reshape(-1,shape[-1]), valid_lens, -1e6)
    return nn.functional.softmax(X.reshape(shape),dim=-1)

In [7]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.4497, 0.5503, 0.0000, 0.0000],
         [0.4146, 0.5854, 0.0000, 0.0000]],

        [[0.2204, 0.3066, 0.4730, 0.0000],
         [0.3412, 0.3344, 0.3244, 0.0000]]])

In [8]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3948, 0.3773, 0.2280, 0.0000]],

        [[0.5193, 0.4807, 0.0000, 0.0000],
         [0.2055, 0.2616, 0.3542, 0.1788]]])

### AdditiveAttention


回顾：注意力机制函数一般可写作$f(x) = \sum_i \alpha(x,x_i)y_i$  
公式中$\alpha(x,x_i)$指的是注意力权重函数  
而$\alpha(x,x_i)$展开来又可以写作$softmax(a(x,x_i))$其中$a(x,x_i)$就是我们所说的注意力分数。具体过程可以用下图来描述：  
<img src="Attention_score.png" width=500px>  
AdditiveAttention就是其中一种attention scoring function设计，用来求出attention weights。  
对于AdditiveAttention来说，可学参数有三个$W_k\in R^{h\times k},W_q \in R^{h\times q},v\in R^{h}$  
完整的函数这样表达：$a(K,q) = v^Ttanh(W_kk + W_qq)$  
其中h是可以调整的超参数，表示隐藏层的大小，这整个函数的目的就是将输入的key和query经过矩阵乘法，变换成长度为h的向量，最后再和$v^T$相乘得到一个attention weight。  
需要注意的是，这里的$v$是一个参数，和key所对应的value不同  
因此，这整个函数其实也等价于将key和query合并起来后放到一个隐藏层大小为h，输出大小为1的但隐藏层NLP  
下面是具体实现（其中包含一些pytorch技巧）：  

In [9]:
class AdditiveAttention(nn.Module):
    def __init__(self, key_size, query_size, num_hiddens, dropout):
        super(AdditiveAttention, self).__init__()
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias = False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
            #按照公式，nn的线性层类实例化三个相应的科学参数tensor，都不需要偏置（bias）
        self.dropout = nn.Dropout(dropout)
            #加入Dropout进行正则化
    def forward(self, queries, keys, values, valid_lens):
        # print("origin queries:")
        # print(queries.shape,"  ",queries)
        # print("origin keys:")
        # print(keys.shape, "  ",keys)
        queries, keys = self.W_q(queries), self.W_k(keys)
            #直接调用线性层的方法进行矩阵计算
        # print("\n\n")
        # print("queries:")
        # print(queries.shape,"  ",queries)
        # print("keys:")
        # print(keys.shape, "  ",keys)
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
            #这里继续利用的pytorch的增加维度方法和广播机制，按照公式将query和key相加，我们马上会具体解释是如何实现的
        # print("unsqueeze queries shape:",queries.unsqueeze(2).shape)
        # print("unsqueeze keys shape:",keys.unsqueeze(1).shape)
        # print("features:")
        # print(features.shape,"  ",features)
        features = torch.tanh(features)
            #使用tanh激活函数对features进行激活
        scores = self.w_v(features).squeeze(-1)
            #最后将我们得到的score压缩为最初的三维，保持维度数不变
        # print("\n\n")
        # print("w_v · features shape:",self.w_v(features).shape)
        # print("w_v · features:",self.w_v(features))
        # print("squeeze shape:",scores.shape)
        self.attention_weights = masked_softmax(scores, valid_lens)
            #进行masked_softmax后得到我们最终的attention_weights
        return torch.bmm(self.dropout(self.attention_weights), values)
            #最后使用bmm方法，按照batch将dropout后的attention_weights和values进行矩阵乘法，得到最后的预测输出。

对于这整个过程来说，最难理解的是`features = queries.unsqueeze(2) + keys.unsqueeze(1)`和`scores = self.w_v(features).squeeze(-1)`这两行代码到底做了一件什么事。下面我们具体举个例子来说明。  
一般来说，我们输入的query和keys每个维度的大小是不同的（除batch_size外，batch_size一般相同），而按照注意力函数的原理，需要将每一个query和所有的key都输入Attention scoring function来得到相应的Attention weights，再和value相乘得到最终的输出（可以参考上面的过程图）  
因此，假设我们输入的`keys.shape=(2,3,6)`,`query.shape=(2,4,6)`（在进行了第一行的矩阵乘法之后，keys和query的张量长度已经相同，都为`h`）,为了将keys和query作加法运算，需要将二者进行一一对应，我们最终需要得到一个大小为`(2,4,3,6)`的矩阵。联想到我们前面在mask函数中的做法，可以利用广播机制来实现。  
`.unsqueeze()`这个方法就是在指定位置（dim）增加一个大小为1的维度。于是，经过`queries.unsqueeze(2)`后`queryies.shape=(2,4,1,6)`即在第2维增加了一个大小为1的维度。相应的，`keys.unsqueeze(1)`后，`keys.shape=(2,1,3,6)`。扩展维度之后，keys中大小为1的维度对应的是quey中的`quey_num`大小，queries中大小为1的维度对应的是keys中的`key_num`大小，将二者相加时，根据广播机制，最终得到的`features.shape=(2,4,3,6)`，这表示着什么？  
对于`features[b,i,j,:]`这个张量，恰好时第`b`个批次中，第`i`个query和第`j`个key向量相加的结果。  
将`features`用`tanh`进行激活之后，`scores = self.w_v(features).squeeze(-1)`这行代码先将`features`与`w_v`进行矩阵乘法（`self.w_v(features)`）。由于线性层（nn.Linear）的矩阵乘法会自动作用在最后一个维度上，因此会将最后长为`6`的张量转化成一个长度为1的标量。因此，对于`features[b,i,j,0]`就代表了第`b`个批次中，第`i`个query和第`j`个key的原始的注意力分数，由于这里最后的0其实是多余的，且我们需要保持三维张量的形状，因此，用`.squeeze(-1)`将最后一个维度移除，最终`features[b,i,j]`就代表了第`b`个批次中，第`i`个query和第`j`个key的原始的注意力分数。

我们可以加以验证：

In [10]:
queries, keys = torch.normal(0, 1, (2, 1, 20)), torch.ones((2, 10, 2))
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(2, 1, 1)
valid_lens = torch.tensor((2, 6))
attention = AdditiveAttention(key_size=2, query_size=20, num_hiddens=8, dropout=0.1)
attention.eval()
attention(queries, keys, values, valid_lens)


tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]], grad_fn=<BmmBackward0>)

相应的中间参数如下（可以自行运行验证）：  
其实就重点看看形状有个印象  
```
origin queries:
torch.Size([2, 1, 20])    
tensor([[[-1.3878, -0.1040, -0.3397,  0.2280,  1.0528,  0.2424,  0.0730,
          -0.1134,  1.3424, -0.4713,  0.0297, -0.2750, -0.5318, -0.3703,
          -0.0506, -0.6353,  1.0778, -1.6444, -0.7185, -0.4319]],

        [[ 0.0506,  1.7803, -0.5501, -1.1848,  1.7455, -0.1357, -0.3519,
          -0.1018, -0.4424, -1.3661, -0.5762,  1.9788,  1.1600,  0.4395,
          -0.8395,  1.1469, -0.2089,  0.2985,  0.2103,  1.4174]]])
origin keys:
torch.Size([2, 10, 2])    
tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])



queries:
torch.Size([2, 1, 8])    
tensor([[[ 0.2826,  0.3332, -0.3560,  0.2125, -0.1740,  0.1085, -0.2761,
           0.6221]],

        [[ 0.0817, -0.0755, -0.0464, -0.3214,  0.3300, -0.9857,  0.8639,
          -0.4190]]], grad_fn=<UnsafeViewBackward0>)
keys:
torch.Size([2, 10, 8])    
tensor([[[-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197]],

        [[-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197],
         [-0.3053, -0.4668, -0.6431,  0.0822, -0.4197, -0.4775, -0.6382,
           0.6197]]], grad_fn=<UnsafeViewBackward0>)
unsqueeze queries shape: torch.Size([2, 1, 1, 8])
unsqueeze keys shape: torch.Size([2, 1, 10, 8])
features:
torch.Size([2, 1, 10, 8])    
tensor([[[[-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418],
          [-0.0227, -0.1336, -0.9991,  0.2947, -0.5938, -0.3690, -0.9143,
            1.2418]]],


        [[[-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007],
          [-0.2236, -0.5423, -0.6895, -0.2391, -0.0897, -1.4631,  0.2257,
            0.2007]]]], grad_fn=<AddBackward0>)



w_v · features shape: torch.Size([2, 1, 10, 1])
w_v · features: 
tensor([[[[0.2425],
          [0.2425],
          [0.2425],
          [0.2425],
          [0.2425],
          [0.2425],
          [0.2425],
          [0.2425],
          [0.2425],
          [0.2425]]],


        [[[0.1433],
          [0.1433],
          [0.1433],
          [0.1433],
          [0.1433],
          [0.1433],
          [0.1433],
          [0.1433],
          [0.1433],
          [0.1433]]]], grad_fn=<UnsafeViewBackward0>)
squeeze shape: torch.Size([2, 1, 10])
```

In [ ]:
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__( **kwargs)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)